In [ ]:
# Question 7
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# ----------- setup -----------
image_path = 'E:/UoM MSc in AI/Semester 3/IT5437 - Computer Vision/Assignment/a1images/a1q8images'

# define small/original pairs
image_pairs = [
    ("im01small.png", "im01.png"),
    ("im02small.png", "im02.png"),
    ("im03small.png", "im03.png"),
    ("taylor_small.jpg", "taylor.jpg"),
    ("taylor_very_small.jpg", "taylor.jpg"),
]

# ----------- helpers -----------
def load_rgb(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def to_gray(img):
    return img if img.ndim == 2 else cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def nssd(img1, img2):
    """Normalized Sum of Squared Differences"""
    a, b = img1.astype(np.float32), img2.astype(np.float32)
    ssd = np.sum((a - b) ** 2)
    denom = np.sum(a**2) + np.sum(b**2) + 1e-9
    return ssd / denom

def zoom_image(img, scale, method="nearest"):
    interp = cv2.INTER_NEAREST if method == "nearest" else cv2.INTER_LINEAR
    H, W = img.shape[:2]
    return cv2.resize(img, (int(W*scale), int(H*scale)), interpolation=interp)

# ----------- main evaluation -----------
scale = 4.0
nearest_scores, bilinear_scores = [], []
pair_names = []

for i, (small_name, orig_name) in enumerate(image_pairs, start=1):
    small_path = os.path.join(image_path, small_name)
    orig_path  = os.path.join(image_path, orig_name)
    if not os.path.exists(small_path) or not os.path.exists(orig_path):
        continue

    small, orig = load_rgb(small_path), load_rgb(orig_path)

    up_nearest = zoom_image(small, scale, "nearest")
    up_bilinear = zoom_image(small, scale, "bilinear")

    # match to original size
    H, W = orig.shape[:2]
    up_nearest = cv2.resize(up_nearest, (W, H), interpolation=cv2.INTER_NEAREST)
    up_bilinear = cv2.resize(up_bilinear, (W, H), interpolation=cv2.INTER_LINEAR)

    # compute errors
    scoreN = nssd(to_gray(up_nearest), to_gray(orig))
    scoreB = nssd(to_gray(up_bilinear), to_gray(orig))

    nearest_scores.append(scoreN)
    bilinear_scores.append(scoreB)
    pair_names.append(f"Pair{i}")

    print(f"{small_name:18s} → {orig_name:12s} | "
          f"Nearest nSSD={scoreN:.5f}, Bilinear nSSD={scoreB:.5f}")

    # --- visualize each case ---
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(orig);        plt.title("Original"); plt.axis("off")
    plt.subplot(1,3,2); plt.imshow(up_nearest);  plt.title(f"Nearest x{scale}\nnSSD={scoreN:.4f}"); plt.axis("off")
    plt.subplot(1,3,3); plt.imshow(up_bilinear); plt.title(f"Bilinear x{scale}\nnSSD={scoreB:.4f}"); plt.axis("off")
    plt.tight_layout(); plt.show()

# ----------- summary histogram -----------
x = np.arange(len(nearest_scores))
plt.figure(figsize=(8,4))
plt.bar(x-0.2, nearest_scores, width=0.4, label="Nearest")
plt.bar(x+0.2, bilinear_scores, width=0.4, label="Bilinear")
plt.xticks(x, pair_names)
plt.ylabel("Normalized SSD")
plt.title("Zoom Quality Comparison (lower = better)")
plt.legend()
plt.show()

# ----------- analysis -----------
print("\n" + "="*60)
print("Analysis of Results")
print("="*60)

for i, name in enumerate(pair_names):
    better = "Bilinear" if bilinear_scores[i] < nearest_scores[i] else "Nearest"
    print(f"{name}: Bilinear nSSD={bilinear_scores[i]:.5f}, "
          f"Nearest nSSD={nearest_scores[i]:.5f} → {better} interpolation is better")

print("\nOverall Trend:")
print("- Bilinear interpolation consistently produces lower nSSD values.")
print("- This means bilinear zoomed images are closer to the original ground truth.")
print("- Nearest-neighbor often introduces blocky artifacts (pixelation).")
print("- Bilinear achieves smoother results, though sometimes slightly blurred.")